In [14]:

from dotenv import load_dotenv, find_dotenv 
load_dotenv()

import pandas as pd
import polars as pl
from financialtools.utils import dataframe_to_json, get_fin_data_year
from financialtools.wrappers import read_financial_results

In [15]:
ticker = 'ISP.MI'
# year = 2024

metrics, eval_metrics, composite_scores, red_flags = read_financial_results(
    ticker=ticker,
    # time=year,
    input_dir='financial_data', 
    sheet_name='sheet1')

metrics

,ticker,time,GrossMargin,OperatingMargin,NetProfitMargin,EBITDAMargin,ROA,ROE,FCFToRevenue,FCFYield,FCFtoDebt,DebtToEquity,CurrentRatio
0,ISP.MI,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ISP.MI,2021,NaN,NaN,0.1911,NaN,0.0039,0.0656,0.3157,0.0712,0.0683,1.5871,NaN
2,ISP.MI,2022,NaN,NaN,0.2031,NaN,0.0045,0.0717,4.6570,1.0349,1.0201,1.6112,NaN
3,ISP.MI,2023,NaN,NaN,0.3039,NaN,0.0080,0.1208,-0.7473,-0.1957,-0.1336,2.2220,NaN
4,ISP.MI,2024,NaN,NaN,0.3063,NaN,0.0093,0.1330,-1.4793,-0.4313,-0.2856,2.2489,NaN


In [16]:
metrics, eval_metrics, composite_scores, red_flags = [
    dataframe_to_json(df)
    for df in [metrics, eval_metrics, composite_scores, red_flags]
]

In [17]:
metrics

'[{"ticker": "ISP.MI", "time": 2020, "GrossMargin": NaN, "OperatingMargin": NaN, "NetProfitMargin": NaN, "EBITDAMargin": NaN, "ROA": NaN, "ROE": NaN, "FCFToRevenue": NaN, "FCFYield": NaN, "FCFtoDebt": NaN, "DebtToEquity": NaN, "CurrentRatio": NaN}, {"ticker": "ISP.MI", "time": 2021, "GrossMargin": NaN, "OperatingMargin": NaN, "NetProfitMargin": 0.1911, "EBITDAMargin": NaN, "ROA": 0.0039, "ROE": 0.0656, "FCFToRevenue": 0.3157, "FCFYield": 0.0712, "FCFtoDebt": 0.0683, "DebtToEquity": 1.5871, "CurrentRatio": NaN}, {"ticker": "ISP.MI", "time": 2022, "GrossMargin": NaN, "OperatingMargin": NaN, "NetProfitMargin": 0.2031, "EBITDAMargin": NaN, "ROA": 0.0045, "ROE": 0.0717, "FCFToRevenue": 4.657, "FCFYield": 1.0349, "FCFtoDebt": 1.0201, "DebtToEquity": 1.6112, "CurrentRatio": NaN}, {"ticker": "ISP.MI", "time": 2023, "GrossMargin": NaN, "OperatingMargin": NaN, "NetProfitMargin": 0.3039, "EBITDAMargin": NaN, "ROA": 0.008, "ROE": 0.1208, "FCFToRevenue": -0.7473, "FCFYield": -0.1957, "FCFtoDebt": -

In [18]:
eval_metrics

'[{"ticker": "ISP.MI", "time": 2020, "bvps": NaN, "fcf_per_share": NaN, "eps": NaN, "P/E": NaN, "P/B": NaN, "P/FCF": NaN, "EarningsYield": NaN, "FCFYield": NaN}, {"ticker": "ISP.MI", "time": 2021, "bvps": 3.6289, "fcf_per_share": 0.3934, "eps": 0.22, "P/E": 24.6227, "P/B": 1.4927, "P/FCF": 13.7709, "EarningsYield": 0.0406, "FCFYield": 0.0712}, {"ticker": "ISP.MI", "time": 2022, "bvps": 3.4769, "fcf_per_share": 5.7146, "eps": 0.23, "P/E": 23.5522, "P/B": 1.558, "P/FCF": 0.9479, "EarningsYield": 0.0425, "FCFYield": 1.0349}, {"ticker": "ISP.MI", "time": 2023, "bvps": 3.6396, "fcf_per_share": -1.0807, "eps": 0.42, "P/E": 12.8976, "P/B": 1.4883, "P/FCF": -5.0123, "EarningsYield": 0.0775, "FCFYield": -0.1957}, {"ticker": "ISP.MI", "time": 2024, "bvps": 3.7087, "fcf_per_share": -2.3817, "eps": 0.4875, "P/E": 11.1128, "P/B": 1.4606, "P/FCF": -2.2744, "EarningsYield": 0.09, "FCFYield": -0.4313}]'

In [19]:
composite_scores

'[{"ticker": "ISP.MI", "time": 2020, "composite_score": 3.0}, {"ticker": "ISP.MI", "time": 2021, "composite_score": 2.94}, {"ticker": "ISP.MI", "time": 2022, "composite_score": 3.12}, {"ticker": "ISP.MI", "time": 2023, "composite_score": 2.32}, {"ticker": "ISP.MI", "time": 2024, "composite_score": 2.32}]'

In [20]:
red_flags

'[{"ticker": "ISP.MI", "time": 2023, "metrics": "FCFtoDebt", "red_flag": "Insufficient Free Cash Flow to cover debt"}, {"ticker": "ISP.MI", "time": 2024, "metrics": "FCFtoDebt", "red_flag": "Insufficient Free Cash Flow to cover debt"}, {"ticker": "ISP.MI", "time": 2023, "metrics": "DebtToEquity", "red_flag": "High Debt-to-Equity (>2)"}, {"ticker": "ISP.MI", "time": 2024, "metrics": "DebtToEquity", "red_flag": "High Debt-to-Equity (>2)"}, {"ticker": "ISP.MI", "time": 2023, "metrics": "FCFtoDebt", "red_flag": "Insufficient Free Cash Flow to cover debt"}, {"ticker": "ISP.MI", "time": 2024, "metrics": "FCFtoDebt", "red_flag": "Insufficient Free Cash Flow to cover debt"}, {"ticker": "ISP.MI", "time": 2023, "metrics": "DebtToEquity", "red_flag": "High Debt-to-Equity (>2)"}, {"ticker": "ISP.MI", "time": 2024, "metrics": "DebtToEquity", "red_flag": "High Debt-to-Equity (>2)"}]'

In [21]:
from financialtools.pydantic_models import StockRegimeAssessment

In [22]:
from financialtools.utils import get_tickers
tickers_sector = get_tickers(columns=['ticker', "sector"])
sector = tickers_sector.filter(pl.col('ticker')==ticker)['sector'][0]
sector

'Finance'

In [23]:
market_metrics = pd.read_excel('financial_data/metrics_by_sectors.xlsx')
market_metrics = market_metrics[market_metrics["sector"] == sector]
market_metrics = market_metrics[market_metrics["time"] == 2024]
market_metrics = market_metrics[['metrics', 'market_value']]
market_metrics

from financialtools.utils import dataframe_to_json

market_metrics = dataframe_to_json(market_metrics)
market_metrics

'[{"metrics": "CurrentRatio", "market_value": 3.583470588235294}, {"metrics": "DebtToEquity", "market_value": 1.206261764705882}, {"metrics": "EBITDAMargin", "market_value": 0.604935294117647}, {"metrics": "FCFToRevenue", "market_value": -0.3163}, {"metrics": "FCFYield", "market_value": -0.05095000000000001}, {"metrics": "FCFtoDebt", "market_value": -0.1077125}, {"metrics": "GrossMargin", "market_value": 0.6767882352941177}, {"metrics": "NetProfitMargin", "market_value": 0.2802777777777778}, {"metrics": "OperatingMargin", "market_value": 0.03461764705882354}, {"metrics": "ROA", "market_value": 0.01228611111111111}, {"metrics": "ROE", "market_value": 0.09369999999999999}]'

In [24]:
eval_market_metrics = pd.read_excel('financial_data/eval_metrics_by_sectors.xlsx')
eval_market_metrics = eval_market_metrics[eval_market_metrics["sector"] == sector]
eval_market_metrics = eval_market_metrics[eval_market_metrics["time"] == 2024]
eval_market_metrics = eval_market_metrics[['eval_metrics', 'market_value']]
eval_market_metrics


eval_market_metrics = dataframe_to_json(eval_market_metrics)
eval_market_metrics

'[{"eval_metrics": "EarningsYield", "market_value": 0.07978214285714288}, {"eval_metrics": "FCFYield", "market_value": -0.05095000000000001}, {"eval_metrics": "P/B", "market_value": 1.575236111111111}, {"eval_metrics": "P/E", "market_value": 14.27628518518519}, {"eval_metrics": "P/FCF", "market_value": -18.83078235294117}, {"eval_metrics": "bvps", "market_value": 9.895380555555555}, {"eval_metrics": "eps", "market_value": 1.253885714285714}, {"eval_metrics": "fcf_per_share", "market_value": -0.1746764705882353}]'

In [25]:
# from financialtools.pydantic_models import StockRegimeAssessment

# from pydantic import BaseModel, Field
# from typing import Literal, List, Dict, Optional
from langchain_core.output_parsers import PydanticOutputParser

from langchain.output_parsers import OutputFixingParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# # Pydantic output model
# class StockRegimeAssessment(BaseModel):
#     regime: Literal["bull", "bear"] = Field(
#         ..., description="The fundamental regime classification of the stock"
#     )
#     rationale: str = Field(
#         ..., description="Concise explanation justifying the regime classification based on the financial metrics, composite ratio and red flags"
#     )
#     metrics_movement: str = Field(
#         ..., description=(
#             "A summary description of how key financial metrics have moved across years, "
#             "e.g., 'GrossMargin increased steadily, DebtToEquity rose sharply, FCFYield remained stable.'"
#         )
#     )
#     non_aligned_findings: Optional[str] = Field(
#         None,
#         description=(
#             "Observations or signals that are not aligned with the overall metric trends, "
#             "such as contradictory indicators, anomalies."
#         )
#     )


# Instantiate the LLM (OpenAI GPT-4 or your preferred model)
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# Instantiate the parser with the Pydantic model
parser = PydanticOutputParser(pydantic_object=StockRegimeAssessment)
# Wrap your parser with OutputFixingParser
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# Get the format instructions string from the parser
format_instructions = parser.get_format_instructions()

system_prompt_template = """
You are a trader assistant specializing in fundamental analysis. 

Based on the following financial data, provide a concise overall assessment that classifies 
the stock’s current fundamental regime as one of:

- bull: Strong and improving fundamentals supporting a positive outlook.
- bear: Weak or deteriorating fundamentals indicating risk or decline.

Financial data constists of financial metrics, evaluation metrics, composite score and red flags.

Financial metrics provided are the following:

Profitability and Margin Metrics:
    -GrossMargin: gross profit / total revenue 
    -OperatingMargin: operating income / total revenue
    -NetProfitMargin: net income / total revenue
    -EBITDAMargin: ebitda / total revenue
Returns metrics:
    -ROA: net income / total assets
    -ROE: net income / total equity
Cash Flow Strength metrics: 
    -FCFToRevenue: free cash flow / total revenue
    -FCFYield: free cash flow / market capitalization
    -FCFToDebt:: free cash flow / total debt
Leverage & Solvency metrics:
    -DebtToEquity: total debt / total equity
Liquidity metrics:
    -CurrentRatio: working capital / total liabilities


Evaluation metrics provided are the following:
    -bvps: total equity / shares outstanding
    -fcf_per_share: free cash flow / shares outstanding
    -eps: earning per share
    -P/E: current stock price / eps
    -P/B: current stock price / bvps
    -P/FCF: current stock price / fcf_per_share
    -EarningsYield: eps / current stock price
    -FCFYield: free cash flow / market capitalization

The composite score is a weighted average (1 to 5) that summarizes the company’s overall fundamental health.
It reflects profitability, efficiency, leverage, liquidity, and cash flow strength, based on the above mentioned financial metrics (evaluation metrics do not kick in the calculation).

The composite score ranges:
1 = Weak fundamentals
5 = Strong fundamentals

Each financial metric (evaluation metric do not kick in in the calculation) is scored on a 1–5 scale and multiplied by its weight. The composite score is the sum of weighted scores divided by the total weight.

A red flag is an early warning signal that highlights potential weaknesses in a company’s financial statements 
or business quality. These warnings do not always mean immediate distress, but they indicate heightened risk that 
traders should carefully consider before taking a position.

Your assessment must take into account also financial and evaluation metrics information from the market. 

"""

# Create a ChatPromptTemplate with system message and user input

system_prompt_filled = system_prompt_template.format(format_instructions=format_instructions)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_filled),
    ("human", "Metrics:\n{metrics}\nScores:\n{scores}\nEvaluation Metrics:\n{eval_metrics}\nRedFlags:\n{red_flags}\nMarket metrics:{market_metrics}\nMarket evaluation metrics:{eval_market_metrics}"),
])

# Create a runnable chain: prompt followed by LLM invocation
chain = prompt | llm | parser


# Then invoke with a dict containing 'financial_data'
response = chain.invoke({
    "metrics": metrics,  
    "eval_metrics": eval_metrics,
    "scores": composite_scores,
    "red_flags": red_flags,   
    "market_metrics": market_metrics, 
    "eval_market_metrics": eval_market_metrics
})


In [26]:
import rich

rich.print(response)

StockRegimeAssessment(
    ticker='XYZ',
    regime='bear',
    regime_rationale='Financial metrics show high leverage with debt-to-equity above 2, negative free cash flow in 
recent years, and low returns on assets and equity, indicating deteriorating financial health and increased risk.',
    metrics_movement='Profit margins improved slightly from 2021 to 2024, but debt levels and negative cash flows 
increased, with FCF to debt ratios remaining below 0, signaling worsening liquidity and leverage issues.',
    non_aligned_findings=None,
    evaluation='undervaluated',
    evaluation_rationale='Despite some margin improvements, high debt, negative free cash flow, and low returns 
suggest the stock is undervalued due to underlying financial distress.',
    market_eval_metrics_comparison='Market valuations are reasonable relative to earnings and book value, but cash 
flow issues are not reflected in current market prices, indicating potential undervaluation.'
)